# [Module 3.1] Use Case Recommender 생성

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 4가지의 Use Case 를 생성

--- 
이 노트북은 약 50분 소요 됩니다.



# 0. 환경 설정

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
# suffix = str(np.random.uniform())[4:9]

# 1. 미리 정의된 Use Case 확인

In [5]:
MostViewedRecipeARN = 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views'
BestSellersRecipeARN = 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases'
FrequentlyBoughtTogetherRecipeARN = 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together'
CustomersWhoViewedXRecipeARN = 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed'


# 2.  Most Viewd Recommender  생성 

In [6]:
import boto3

personalize = boto3.client('personalize')

create_recommender_response = personalize.create_recommender(
  name = 'Recommender-MostViewed-' + suffix,
  recipeArn = MostViewedRecipeARN,
  datasetGroupArn = dataset_group_arn,
)



In [7]:
MostViewed_RecommenderARN = create_recommender_response['recommenderArn']
print('MostViewed_RecommenderARN :' + MostViewed_RecommenderARN )

MostViewed_RecommenderARN :arn:aws:personalize:ap-northeast-2:057716757052:recommender/Recommender-MostViewed-96279


# 3.  BestSellers Recommender 생성 

In [8]:
import boto3

personalize = boto3.client('personalize')

create_recommender_response = personalize.create_recommender(
  name = 'Recommender-BestSeller-' + suffix,
  recipeArn = BestSellersRecipeARN,
  datasetGroupArn = dataset_group_arn,
)




In [9]:
BestSeller_RecommenderARN = create_recommender_response['recommenderArn']
print('BestSeller_RecommenderARN :' + BestSeller_RecommenderARN )

BestSeller_RecommenderARN :arn:aws:personalize:ap-northeast-2:057716757052:recommender/Recommender-BestSeller-96279


# 4.  FrequentlyBoughtTogether Recommender 생성 

In [10]:
import boto3

personalize = boto3.client('personalize')

create_recommender_response = personalize.create_recommender(
  name = 'Recommender-FrequentlyBoughtTogether-' + suffix,
  recipeArn = FrequentlyBoughtTogetherRecipeARN,
  datasetGroupArn = dataset_group_arn,
)





In [11]:
FrequentlyBoughtTogether_RecommenderARN = create_recommender_response['recommenderArn']
print('FrequentlyBoughtTogether_RecommenderARN :' + FrequentlyBoughtTogether_RecommenderARN )

FrequentlyBoughtTogether_RecommenderARN :arn:aws:personalize:ap-northeast-2:057716757052:recommender/Recommender-FrequentlyBoughtTogether-96279


# 5. CustomersWhoViewedX Recommender 생성 

In [12]:
import boto3

personalize = boto3.client('personalize')

create_recommender_response = personalize.create_recommender(
  name = 'Recommender-CustomersWhoViewedX-' + suffix,
  recipeArn = CustomersWhoViewedXRecipeARN,
  datasetGroupArn = dataset_group_arn,
)

In [13]:
CustomersWhoViewedX_RecommenderARN = create_recommender_response['recommenderArn']
print('CustomersWhoViewedX_RecommenderARN :' + CustomersWhoViewedX_RecommenderARN )

CustomersWhoViewedX_RecommenderARN :arn:aws:personalize:ap-northeast-2:057716757052:recommender/Recommender-CustomersWhoViewedX-96279


# 6. 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [15]:
%%time


max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:

    # status_most_viewed
    describe_recommender_response = personalize.describe_recommender(
        recommenderArn = MostViewed_RecommenderARN
    )  
    status_most_viewed = describe_recommender_response["recommender"]["status"]
    print("status_most_viewed : {}".format(status_most_viewed))
    
    # status_best_seller
    describe_recommender_response = personalize.describe_recommender(
        recommenderArn = BestSeller_RecommenderARN
    )  
    status_best_seller = describe_recommender_response["recommender"]["status"]
    print("status_best_seller : {}".format(status_best_seller))
    
    # status_frequently_bought_t
    describe_recommender_response = personalize.describe_recommender(
        recommenderArn = FrequentlyBoughtTogether_RecommenderARN
    )  
    status_frequently_bought_t = describe_recommender_response["recommender"]["status"]
    print("status_frequently_bought_t : {}".format(status_frequently_bought_t))

    # status_customer_who_view
    describe_recommender_response = personalize.describe_recommender(
        recommenderArn = CustomersWhoViewedX_RecommenderARN
    )  
    status_customer_who_view = describe_recommender_response["recommender"]["status"]
    print("status_customer_who_view : {}".format(status_customer_who_view))
        
    if (status_most_viewed == "ACTIVE" or status_most_viewed == "CREATE FAILED") &\
       (status_best_seller == "ACTIVE" or status_best_seller == "CREATE FAILED") &\
       (status_frequently_bought_t == "ACTIVE" or status_frequently_bought_t == "CREATE FAILED") &\
       (status_customer_who_view == "ACTIVE" or status_customer_who_view == "CREATE FAILED") :
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All recommenders completed")

status_most_viewed : CREATE IN_PROGRESS
status_best_seller : CREATE IN_PROGRESS
status_frequently_bought_t : CREATE IN_PROGRESS
status_customer_who_view : CREATE IN_PROGRESS
-------------------------------------->
status_most_viewed : CREATE IN_PROGRESS
status_best_seller : CREATE IN_PROGRESS
status_frequently_bought_t : CREATE IN_PROGRESS
status_customer_who_view : CREATE IN_PROGRESS
-------------------------------------->
status_most_viewed : CREATE IN_PROGRESS
status_best_seller : CREATE IN_PROGRESS
status_frequently_bought_t : CREATE IN_PROGRESS
status_customer_who_view : CREATE IN_PROGRESS
-------------------------------------->
status_most_viewed : CREATE IN_PROGRESS
status_best_seller : CREATE IN_PROGRESS
status_frequently_bought_t : CREATE IN_PROGRESS
status_customer_who_view : CREATE IN_PROGRESS
-------------------------------------->
status_most_viewed : CREATE IN_PROGRESS
status_best_seller : CREATE IN_PROGRESS
status_frequently_bought_t : CREATE IN_PROGRESS
status_customer_

# 7. Recommender 변수 저장

In [16]:

%store MostViewed_RecommenderARN
%store BestSeller_RecommenderARN

%store FrequentlyBoughtTogether_RecommenderARN
%store CustomersWhoViewedX_RecommenderARN



Stored 'MostViewed_RecommenderARN' (str)
Stored 'BestSeller_RecommenderARN' (str)
Stored 'FrequentlyBoughtTogether_RecommenderARN' (str)
Stored 'CustomersWhoViewedX_RecommenderARN' (str)
